In [1]:
import glob
import json
import re
import os
from os import path


In [2]:
gpu = False
import tensorflow as tf
if gpu:
    from keras.backend.tensorflow_backend import set_session
    physical_devices = tf.config.list_physical_devices('GPU') 
    try: 
      tf.config.experimental.set_memory_growth(physical_devices[0], True) 
    except: 
      # Invalid device or cannot modify virtual devices once initialized. 
      pass # dynamically grow GPU memory 


In [3]:
def clean_images(basepath):
    count = 0
    images = glob.glob(f'{basepath}/**/*.jpg')
    for image in images:
        os.remove(image)
        count += 1
    print("Cleaned ", count, " images.")
    audioFiles = glob.glob(f'{basepath}/**/*.wav')
    for wav in audioFiles:
        os.remove(wav)

In [4]:
clean_images("train")
clean_images("test")
clean_images("validation")

Cleaned  0  images.
Cleaned  0  images.
Cleaned  0  images.


In [5]:
def fileExists(basepath, file):
    return path.exists(f'{basepath}/{file}')

In [6]:
import numpy as np
from scipy.spatial.distance import euclidean
import librosa

from fastdtw import fastdtw

def isAudioFake(basepath, fakeMP4, originalMP4):
    if fileExists(basepath, fakeMP4) and fileExists(basepath, originalMP4):
        fakeAudio = re.sub(r'(\.mp4$)', '.wav', fakeMP4)
        if not fileExists(basepath, fakeAudio):
            !ffmpeg -i {basepath}/{fakeMP4} -ab 160k -ac 2 -ar 44100 -vn {basepath}/{fakeAudio}
        originalAudio = re.sub(r'(\.mp4$)', '.wav', originalMP4)
        if not fileExists(basepath, originalAudio):
            !ffmpeg -i {basepath}/{originalMP4} -ab 160k -ac 2 -ar 44100 -vn {basepath}/{originalAudio}
    
        y1, sr1 = librosa.load(f'{basepath}/{fakeAudio}')
        y2, sr2 = librosa.load(f'{basepath}/{originalAudio}')
        mfcc1 = librosa.feature.mfcc(y1,sr1)   #Computing MFCC values
        mfcc2 = librosa.feature.mfcc(y2, sr2)
        distance, path = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)
        print("distance = ", distance)
        return distance > 0
    else:
        return False


In [7]:
def process_movie(basepath, moviePath, label, split):
    if fileExists(basepath, moviePath):
        print("Processing movie, ", moviePath, " with label, ", label, " and split, ", split)
        movie = re.sub(r'(\.mp4$)', '', moviePath)
        !ffmpeg -i {basepath}/{moviePath} -r 1/1 {split}/{label}/{movie}%08d.jpg
        return 1
    return 0

In [8]:
#metadataFiles = glob.glob('G:/deepfake/**/**/metadata.json')
metadataFiles = glob.glob('movies/**/metadata.json')
MAX_MOVIES=120

moviesProcessed = 0
testSamplingRate = 5  # every 5th movie, put into test, every sixth movie put into validation

def processMetadataFile(metadataFile, moviesProcessed):
    numRealMovies = 0
    numFakeMovies = 0
    numAudioFakes = 0
    originalsProcessed = 0

    for metadata in metadataFile:
        print("found metadata, ", metadata)
        basepath=re.sub(r'(metadata.json$)', '', metadata)
        with open(metadata) as f:        
            data = json.load(f)
            for key in data:
                if moviesProcessed > MAX_MOVIES:
                    print("moviesProcessed > MAX")
                    break
                label = data[key]['label']
                if moviesProcessed % testSamplingRate == 0:
                    split = "test"
                elif moviesProcessed % testSamplingRate == 1:
                    split = "validation"
                else:
                    split = "train"
                if label == "FAKE" and 2 * numFakeMovies > numRealMovies:
                    print("Skipping because we need balanced fake and real videos")
                elif label == "FAKE":
                    original = data[key]['original']
                    
                    if isAudioFake(basepath, key, original):
                        numAudioFakes += 1
                    else:
                        processed = process_movie(basepath, key, label, split)
                        moviesProcessed += processed
                        numFakeMovies += processed

                        processed = process_movie(basepath, original, "REAL", split)
                        moviesProcessed += processed
                        numRealMovies += processed
                        originalsProcessed += processed

                else:
                    numRealMovies += 1
                    processed = process_movie(basepath, key, label, split)

                    moviesProcessed += processed
    print("Processed originals", originalsProcessed, " videos for train.")
    print("Detected and skipped ", numAudioFakes, " fake videos with fake audio.")

    return moviesProcessed
      
moviesProcessed += processMetadataFile(metadataFiles, moviesProcessed)
    



found metadata,  movies/dfdc_train_part_0/metadata.json
distance =  0.0
Processing movie,  htorvhbcae.mp4  with label,  FAKE  and split,  test
ffmpeg version 4.2.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.8)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.1_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-fre

[swscaler @ 0x1115eb000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'train/FAKE/fckxaqjbxk%08d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
    Stream #0:0(und): Video: mjpeg, yuvj420p(pc), 1080x1920 [SAR 1:1 DAR 9:16], q=2-31, 200 kb/s, 1 fps, 1 tbn, 1 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      encoder         : Lavc58.54.100 mjpeg
    Side data:
      cpb: bitrate max/min/avg: 0/0/200000 buffer size: 0 vbv_delay: -1
frame=   12 fps=6.1 q=22.8 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed=6.14x    
video:1200kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake an

ffmpeg version 4.2.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.8)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.1_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libs

frame=   12 fps=5.7 q=21.9 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed=5.72x    
video:1011kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced f

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'movies/dfdc_train_part_0//gilzdnbpep.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.71.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 7263 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1080x1920 [SAR 1:1 DAR 9:16], 7194 kb/s, 29.97 fps, 29.97 tbr, 299691 tbn, 59.94 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, mono, fltp, 69 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:1 -> #0:0 (aac (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to 'movies/dfdc_train_part_0//gilzdnbpep.wav':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    ISFT            : Lavf58.29.100
    Stream #0:0(und): Audio: pcm_s16

...

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'movies/dfdc_train_part_0//uprwuohbwx.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.71.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 12926 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1080x1920, 12865 kb/s, 29.97 fps, 29.97 tbr, 29969 tbn, 59.94 tbc (default)
    Metadata:
      handler_name    : VideoHandler
    Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, mono, fltp, 69 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> mjpeg (native))
Press [q] to stop, [?] for help
[swscaler @ 0x110bcf000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'train/REAL/uprwuohbwx%08d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    enco

Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> mjpeg (native))
Press [q] to stop, [?] for help
[swscaler @ 0x10d92b000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'train/FAKE/vvldcncxda%08d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
    Stream #0:0(und): Video: mjpeg, yuvj420p(pc), 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 200 kb/s, 1 fps, 1 tbn, 1 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      encoder         : Lavc58.54.100 mjpeg
    Side data:
      cpb: bitrate max/min/avg: 0/0/200000 buffer size: 0 vbv_delay: -1
frame=   12 fps=7.9 q=22.3 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed= 7.9x    
video:1138kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Processing movie,  hicjuubiau.mp4  with label,  REAL  and split,  train
ffmpeg version 4.2.1 Copyright

frame=   12 fps=8.0 q=22.2 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed=7.98x    
video:1038kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Processing movie,  yxvmusxvcz.mp4  with label,  REAL  and split,  validation
ffmpeg version 4.2.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.8)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.1_2 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable

Output #0, image2, to 'train/FAKE/iojmxrzvxe%08d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
    Stream #0:0(und): Video: mjpeg, yuvj420p(pc), 1080x1920 [SAR 1:1 DAR 9:16], q=2-31, 200 kb/s, 1 fps, 1 tbn, 1 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      encoder         : Lavc58.54.100 mjpeg
    Side data:
      cpb: bitrate max/min/avg: 0/0/200000 buffer size: 0 vbv_delay: -1
frame=   12 fps=6.3 q=21.6 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed=6.27x    
video:935kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Processing movie,  aayrffkzxn.mp4  with label,  REAL  and split,  train
ffmpeg version 4.2.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.8)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.1_2 --enable-shared --enable-pthreads --enable

[swscaler @ 0x1059ee000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'train/FAKE/cefpmcqwex%08d.jpg':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
    Stream #0:0(und): Video: mjpeg, yuvj420p(pc), 1080x1920 [SAR 1:1 DAR 9:16], q=2-31, 200 kb/s, 1 fps, 1 tbn, 1 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      encoder         : Lavc58.54.100 mjpeg
    Side data:
      cpb: bitrate max/min/avg: 0/0/200000 buffer size: 0 vbv_delay: -1
frame=   12 fps=6.5 q=21.6 Lsize=N/A time=00:00:12.00 bitrate=N/A dup=0 drop=288 speed= 6.5x    
video:939kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Processing movie,  uqtqhiqymz.mp4  with label,  REAL  and split,  train
ffmpeg version 4.2.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.8)
  config

In [ ]:
print("Processed ", moviesProcessed, " videos for train.")

In [ ]:
from detect_face import crop_face_save_jpg

In [ ]:
trainImages = glob.glob('train/**/*.jpg')


In [ ]:
# Find the faces in each image, and crop it out
# If the face can't be detected, then we remove the image from dataset
for image in trainImages:
    crop_face_save_jpg(image)

In [ ]:
testImages = glob.glob('test/**/*.jpg')
for image in testImages:
    crop_face_save_jpg(image)

In [ ]:
testImages = glob.glob('validation/**/*.jpg')
for image in testImages:
    crop_face_save_jpg(image)

In [ ]:
print("Finished preprocessing movies")

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
import pandas as pd
from pickle import dump

def extract_features(filepath):
    """takes the cropped faces and extracts features from them. takes the file path of the face data"""
    # load model
    model = VGG16()
    # remove the output layer
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = np.zeros((1, 4096))
    filepath = os.path.join(filepath,'*.jpg')
    faces = glob.glob(filepath)
    for face in faces:
        # load an image from file
        image = load_img(face, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        #print(image.shape)
        #prepare the image for the VGG model
        image = preprocess_input(image)
        # get extracted features
        features = np.append(features,  model.predict(image), axis = 0)
        #print(features.shape)
        # save to file
        #dump(features, f)
    #save to pd dataframe
    features = np.delete(features, 0, 0)
    return pd.DataFrame(features)

In [ ]:
model = VGG16()

In [ ]:
train_fake_df = extract_features('train/fake/')
train_real_df = extract_features('train/real/')

test_real_df = extract_features('test/real/')
test_fake_df = extract_features('test/fake/')

In [ ]:
validation_real_df = extract_features('validation/real/')
validation_fake_df = extract_features('validation/fake/')

In [ ]:
# write extracted features to disk
train_fake_df.to_csv('train_fake.csv')
train_real_df.to_csv('train_real.csv')
test_fake_df.to_csv('test_fake.csv')
test_real_df.to_csv('test_real.csv')

In [ ]:
validation_fake_df.to_csv('validation_fake.csv')
validation_real_df.to_csv('validation_real.csv')

In [ ]:
#model = VGG16()

In [ ]:
#model.layers.pop()
#model.layers.pop()
#model.layers.pop()

#model.summary()